In [1]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from google.cloud import bigquery
import json

client = MongoClient('localhost', 27017)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
bq_client = bigquery.Client()

db = client.worldcup

load_dotenv()

tweepy_client = tweepy.Client(os.getenv('BEARER_TOKEN'), wait_on_rate_limit=True)

In [28]:
query = '#JPN #HRV -is:retweet'

tweets = tweepy.Paginator(tweepy_client.search_recent_tweets, 
                        query=query,
                        tweet_fields = ['author_id', 'created_at', 'source', 'lang', 'geo', 'entities'], 
                        user_fields = ['name','username','location','verified'], 
                        expansions = ['geo.place_id', 'author_id'], 
                        place_fields = ['country','country_code']
                        )

# for tweet in tweets:
#     print (tweet.includes)
#     print (type(tweet.includes))
#     break



for tweet in tweets.flatten():
    try:
        print(tweet.data['entities'])
        for key, value in tweet.data.items():
            print (key, value)
        # db.a.insert_one(tweet)
        break
    except ValueError as e:
        print(e)

{'hashtags': [{'start': 0, 'end': 4, 'tag': 'JPN'}, {'start': 5, 'end': 9, 'tag': 'HRV'}]}
entities {'hashtags': [{'start': 0, 'end': 4, 'tag': 'JPN'}, {'start': 5, 'end': 9, 'tag': 'HRV'}]}
source Twitter for Android
edit_history_tweet_ids ['1601715690377207808']
author_id 32342156
id 1601715690377207808
lang qht
created_at 2022-12-10T23:09:13.000Z
text #JPN #HRV


In [ ]:
search_query = "#JPN #HRV"
no_of_tweets = 1

tweets = tweepy_client.search_recent_tweets(query=search_query)


# i=0
for tweet in tweets:
    try:
        # content = {
        #     "Id": tweet.id,
        #     "Location": tweet.user.location,
        #     "Date_Created": tweet.created_at,
        #     "Tweet": tweet.text,
        #     "Reply_to": tweet.in_reply_to_user_id,
        #     "Lang": tweet.lang,
        #     "No_of_Likes": tweet.public_metrics.like_count,
        #     "No_of_Replies": tweet.public_metrics.reply_count,
        #     "No_of_retweets": tweet.public_metrics.retweet_count,
        #     "Source_of_Tweet": tweet.source
        # }
        print(tweet)
        print(type(tweet))
        break
        # db.t.insert_one(tweet)
        print(i)
    except ValueError as e:
        print(e)

In [ ]:
jpn_vs_hrv = bq_client.get_table('tecky-capstone-project.worldcup.jpn_hrv')

x = db.jh.find()
for row in x:
    row['_id'] = str(row['_id'])
    row['Date_Created'] = str(row['Date_Created'])
    bq_client.insert_rows_json(jpn_vs_hrv,[row])